In [195]:
import os
import pandas as pd

rbp31_path = "/home/mrkvrbl/Diplomka/Data/rbp31"

In [196]:
for root, dirs, files in os.walk(rbp24_path):
    if len(files) == 1:
        negatives_path = root + "/" + files[0]
        positives_path = root + "/" + files[1]
        out_file = root + "/sequences.tsv.gz"
        print(out_file)

        negatives_df = pd.read_csv(negatives_path, sep='\t', names=['name', 'seq'], compression='gzip')
        negatives_df['label'] = 0

        positives_df = pd.read_csv(positives_path, sep='\t', names=['name', 'seq'], compression='gzip')
        positives_df['label'] = 1

        df = pd.concat([negatives_df, positives_df], ignore_index=True).sample(frac=1).reset_index(drop=True)
        df.to_csv(out_file, sep='\t', compression='gzip')

        os.remove(negatives_path)
        os.remove(positives_path)

        print('root')
        break

NameError: name 'rbp24_path' is not defined

In [ ]:
df

,name,seq,label
0,chrX:149879841-149879969(),AAACCATAAAACTAGTAAGAACCTCCAGTGATTCTTTTCTGATTAT...,0
1,chr17:55335678-55335806(),ATTAGGTCAGCCCCACCATGAGTTAGATTCCAAGACGGTAGGTTGC...,1
2,chr14:53562752-53562880(),TAATTTCTATGCTTTAAATGCAACTGAATACTATAAGCCAGGAATA...,1
3,chr3:172348543-172348671(),TTAAAATCATTCATAAAATCTACCTTCCCCTTCGTAGTGGGAAATA...,1
4,chr20:33820052-33820180(),actaatcttaggaaatgcctagccctatagtatggtattgtttgaa...,0
...,...,...,...
995,chr7:56064249-56064377(),CCGCTGGAACACGCTCCCTGCCCAGATCAGTGTAGTGCGAGATAAA...,0
996,chrX:70600594-70600722(),tgttggcgcgatcttggctcactgcaacatctgcctcccaggttga...,0
997,chr8:141536234-141536362(),AATGTATTATTTGCATGACAGTCGGTGTTGAAGCACCTTTGTGATT...,1
998,chr17:77767060-77767188(),TTCAAATAGAGAACACTGCGAATCGAAAAGACCTATAAATTATAGC...,0


In [ ]:
def concat_df(positives, negatives):
    """
    Concantenates two dfs
    """
    return pd.concat([positives, negatives], ignore_index=True).sample(frac=1).reset_index(drop=True)


def shuffle(df):
    """
    for each sequence in dataset, shuffle nucleotides in given sequence
    """
    return df['seq'].map(lambda seq: ''.join(random.sample(seq, k=len(seq))))


def shuffle_both(positives, negatives):
    """
    concatenate positives and negatives to form train df, than shuffle sequences in whole df
    """
    df = concat_df(positives, negatives)
    df['seq'] = shuffle(df)
    return df


def neg_as_shuflled_pos(positives, negatives):
    """
    shuffle positives and asign it to negatives, concatanted both to df
    """
    positives_unshufled = positives.copy(deep=True)
    negatives['seq'] = shuffle(positives)
    negatives.dropna(subset=["seq"], inplace=True)
    df = concat_df(positives_unshufled, negatives)
    return df

def to_tsv(df):
    df.to_csv()


In [ ]:
import numpy as np
import pandas as pd
import random
import os
from pathlib import Path


rbp31_path = "/home/mrkvrbl/Diplomka/Data/rbp31"
rbp24_path = "/home/mrkvrbl/Diplomka/Data/rbp24"
count=0

for root, dirs, files in os.walk(rbp24_path):
    if root.endswith('train'):
        for file in files:
            if file.endswith(".tsv.gz"):
                original = pd.read_csv(root + '/' + file, delimiter='\t', index_col=0, compression='gzip')
                
                positives = original[original.label == 1]
                negatives = original[original.label == 0]

                shuffled = shuffle_both(positives, negatives)
                sameGC = neg_as_shuflled_pos(positives, negatives)

                shuffled.to_csv(root + '/shuffled.tsv.gz', sep='\t', compression='gzip')
                sameGC.to_csv(root + '/sameGC.tsv.gz', sep='\t', compression='gzip')


        break

/home/mrkvrbl/miniconda3/envs/diplomka/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/mrkvrbl/miniconda3/envs/diplomka/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
import numpy as np
import pandas as pd
import random
import os
import pybedtools
from pathlib import Path

rbp31_path = "/home/mrkvrbl/Diplomka/Data/rbp31"
rbp24_path = "/home/mrkvrbl/Diplomka/Data/rbp24"
transcripts_path_bed = "/home/mrkvrbl/Diplomka/Data/outputs/random_transcripts.bed"
transcripts_path_csv = "/home/mrkvrbl/Diplomka/Data/outputs/random_transcripts.csv"

introns_bed = "/home/mrkvrbl/Diplomka/Data/reference/region_types/introns.bed.gz"
exons_bed = "/home/mrkvrbl/Diplomka/Data/reference/region_types/exons.bed.gz"
utr5_bed = "/home/mrkvrbl/Diplomka/Data/reference/region_types/5utrs.bed.gz"
utr3_bed = "/home/mrkvrbl/Diplomka/Data/reference/region_types/3utrs.bed.gz"

introns_tsv = "/home/mrkvrbl/Diplomka/Data/outputs/intersections/region_types/introns_intersections.tsv"
exons_tsv = "/home/mrkvrbl/Diplomka/Data/outputs/intersections/region_types/exons_intersections.tsv"
utr5_tsv = "/home/mrkvrbl/Diplomka/Data/outputs/intersections/region_types/utr5_intersections.tsv"
utr3_tsv = "/home/mrkvrbl/Diplomka/Data/outputs/intersections/region_types/utr3_intersections.tsv"

regions_bed = [introns_bed, exons_bed, utr5_bed, utr3_bed]
regions_tsv = [introns_tsv, exons_tsv, utr5_tsv, utr3_tsv]


def intersection(a, b):
    a = pybedtools.BedTool(a)
    b = pybedtools.BedTool(b)

    return a.intersect(b, u=True)


def generate_distributed_sequences(sequences_len, sequences_bed, regions_bed, regions_tsv):
    region_counts = []
    sequences = []

    for region in regions_bed:
        intersect = intersection(sequences_bed, region)
        region_counts.append(len(intersect))

    regions_total_count = sum(region_counts)
    region_ratios = [count / regions_total_count for count in region_counts]
    region_final_counts = [int(sequences_len * ratio) for ratio in region_ratios]
    
    for i in range(4):
        df = pd.read_csv(regions_tsv[i], delimiter="\t", names=['id', 'seq']).sample(n=region_final_counts[i])
        sequences.append(df)

    return pd.concat(sequences, ignore_index=True).sample(frac=1)

def pick_random_positives(negatives_df, positives_len, positives_bed, regions_bed, regions_tsv):
    
    positives_df = generate_distributed_sequences(positives_len, positives_bed, regions_bed, regions_tsv)
    positives_df['label'] = 1

    random_pos_df = pd.concat([negatives_df, positives_df], ignore_index=True).sample(frac=1).reset_index(drop=True)
    return random_pos_df

def pick_distributed_negatives(positives_df, negatives_len, positives_bed, regions_bed, regions_tsv):

    negatives_df = generate_distributed_sequences(negatives_len, positives_bed, regions_bed, regions_tsv)
    negatives_df['label'] = 0

    distributed_neg_df = pd.concat([positives_df, negatives_df], ignore_index=True).sample(frac=1).reset_index(drop=True)
    return distributed_neg_df


for root, dirs, files in os.walk(rbp31_path):
    if root.endswith('test'):
        positives_bed = root + "/positives.bed"
        negatives_bed = root + "/negatives.bed"
 
        original_df = pd.read_csv(root + "/original.tsv.gz", delimiter='\t', index_col=0, compression='gzip')

        print(root)

        negatives_df = original_df[original_df.label == 0]
        negatives_len = len(negatives_df)
        positives_df = original_df[original_df.label == 1]
        positives_len = len(positives_df)

        random_pos_df = pick_random_positives(negatives_df, positives_len, positives_bed, regions_bed, regions_tsv)
        distributed_neg_df = pick_distributed_negatives(positives_df, negatives_len, positives_bed, regions_bed, regions_tsv)

        random_pos_df.to_csv(root + "/random_positives.tsv.gz", sep="\t", compression="gzip")
        distributed_neg_df.to_csv(root + "/distributed_negatives.tsv.gz", sep="\t", compression="gzip")

        os.remove(positives_bed)
        os.remove(negatives_bed)
        break

/home/mrkvrbl/Diplomka/Data/rbp31/CLIPSEQ_elF4AIII_2/test


In [ ]:
original_df = pd.read_csv(original, delimiter='\t', index_col=0, compression='gzip')
original_df

,id,seq,label
0,chr12:27060116-27060244(),agttagaagattcttacttcctcattacaagacttaatttgaagtg...,0
1,chr11:65367062-65367190(),CGCGTCCTGGGCCTCCCGGCGGCTGCAGGTCGCGGCCAAGGCCCAG...,0
2,chr3:151595942-151596070(),ttgaaagggaatggggtgtcgagatgtcaaaacacttaatgagatg...,0
3,chr5:54405142-54405270(),TTTTGGAACCCACATGCAGCATCCTAAGGCCACGGCAGCATCCCAA...,0
4,chr17:48456471-48456599(),CAGAATCCTCCAAGAAGAGGGAAACAGGGAGCAAATAAACAGACCA...,0
...,...,...,...
9995,chr8:42783743-42783871(),CATATACTGTCTCCAGTGTCATGAGTAAGGCAGTGGTAGTATATGG...,0
9996,chr7:99690886-99691014(),CAGCTTTGTCCTTCACTAAACCACTCACCTAGCTGTCTGCCCCTTG...,1
9997,chr3:156866598-156866726(),TTAGCAATTTTTATAGTGTCCTAGAGCTATGCTTTTTATTAAGTAT...,0
9998,chr1:171181114-171181242(),CAGAGAGTAGCCCATGAAATTTCCCATTCTTCAAGGACAAATTCCT...,0


In [ ]:
random_pos_df

,id,seq,label
0,chrX:21674272-21674400(),TCGTAGCGCTCGATCTCAGAGAAGGGCTCGTATCGCCCCAGAAAAG...,0
1,chr11:62380633-62380761(),CAGGGTATTGGGGTCAGGGTGGCATTAGCCCAGCTCAAGCCGGGCC...,0
2,chr8:145140796-145140924(),TGCCCTGCTGGTGCTGACCAGCCCGGCAGCCACGCTCCTTGGCAGC...,0
3,chr9:107539126-107539254(),taagtacttgttgactgaattaaCATATGAGATCTGTGAGTCTGAA...,0
4,chr22:39161095-39161223(),ggcggtgggcgactgtaatccagctactcgggaggctgaggcatga...,0
...,...,...,...
9993,chr22:18200939-18201067(),GAAAAGAAGATTGCTTACATTAGAGTAATACCAGAAAATGTTGATG...,0
9994,chr21:31743796-31743924(),TTGTTTCAAAATTATAGACATTTGACTCTATATTTTGCTCATTCAT...,0
9995,chr5:122275942-122276070(),cctcccaaagtgctagagttacaggcatgagccaccgcgcccagAC...,0
9996,chr1:42935517-42935645(),ttgatgggagaggtaaaagttttgaatagttgctgagtgaaatagt...,0


In [ ]:
distributed_neg_df

,id,seq,label
0,chr11:799570-799698(),GGCAGAGGATGGGCGACAGAGGGGTCCTGTCCACCTGCCCAGGACC...,0
1,chr16:5075531-5075659(),GAGGCTCCCCGTTCATCTCCTGCAGCGGGTGGTATGCATAGTCCCC...,0
2,chr12:58089514-58089642(),AAGATGTCTGCACTCCATCCCCCTGATTGTTTATTTTTTAATTGTC...,0
3,chr4:104027356-104027484(),CAGAGAAGTGACAAAGAGGAGTCTACTGAGTTTTGCACTCAGGCAC...,0
4,chr13:93879781-93879909(),GTGAAGGCTCGGAGCTGCGGAGAGGTCCGCCAGGCGTACGGTGCCA...,0
...,...,...,...
9992,chr1:1470673-1470801(),GAGGTCCCTTTTACATCTGTACAGCAGCCGGTTGGCTCGGTGGCCC...,0
9993,chr17:48640407-48640535(),TCTCCCTTGTAGCCCCTGTGAGCCTAGCAGGGACTCTGCTTTGGGG...,0
9994,chr4:151773702-151773830(),GTCAGAAGATACTTCTAAATCATCTGCATTCCTTGTCTCATTTGTC...,1
9995,chr3:137981468-137981596(),ccctgaccctggagaatctaatttaattggtctgctgtatggcctg...,0


In [ ]:
import numpy as np
import pandas as pd
import random
import os
from pathlib import Path


rbp31_path = "/home/mrkvrbl/Diplomka/Data/rbp31"
rbp24_path = "/home/mrkvrbl/Diplomka/Data/rbp24"


def concat_df(positives, negatives):
    """
    Concantenates two dfs
    """
    return pd.concat([positives, negatives], ignore_index=True).sample(frac=1).reset_index(drop=True)


def shuffle(df, n_samples):
    """
    for each sequence in dataset, shuffle nucleotides in given sequence
    """
    return df['seq'].map(lambda seq: ''.join(random.sample(seq, k=len(seq)))).sample(n=n_samples)


def shuffle_both(positives, negatives):
    """
    concatenate positives and negatives to form train df, than shuffle sequences in whole df
    """
    df = concat_df(positives, negatives)
    df['seq'] = shuffle(df, len(df))
    return df


def neg_as_shuflled_pos(positives, negatives):
    """
    shuffle positives and asign it to negatives, concatanted both to df
    """
    positives_unshufled = positives.copy(deep=True)
    negatives.loc[:,'seq'] = list(shuffle(positives, len(negatives)))
    df = concat_df(positives_unshufled, negatives)
    return df, seqs, negatives

def to_tsv(df):
    df.to_csv()


#uncoment for rbp24
for root, dirs, files in os.walk(rbp24_path):
    if root.endswith('train'):
        original = pd.read_csv(root + '/original.tsv.gz', delimiter='\t', index_col=0, compression='gzip')
                
        positives = original[original.label == 1]
        negatives = original[original.label == 0]

        shuffled = shuffle_both(positives, negatives)
        sameGC, seqs, negatives = neg_as_shuflled_pos(positives, negatives)

        shuffled.to_csv(root + '/shuffled.tsv.gz', sep='\t', compression='gzip')
        sameGC.to_csv(root + '/sameGC.tsv.gz', sep='\t', compression='gzip')

        break


/home/mrkvrbl/miniconda3/envs/diplomka/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:
import os
import pandas as pd
from utils.utils import gc_count

rbp31_path = "/home/mrkvrbl/Diplomka/Data/rbp31"
rbp24_path = "/home/mrkvrbl/Diplomka/Data/rbp24"

test_dfs = {}
content = []


for root, dirs, files in os.walk(rbp24_path):
    if len(dirs) == 2:
        protein_name = root.split("/")[-1]

    if root.endswith('test'):
        for file in files:
            if file.endswith("tsv.gz"):
                test_dfs[file.split(".")[0]] = pd.read_csv(root + "/" + file, delimiter="\t", index_col=0, compression="gzip")

        for data_type, test_df in test_dfs.items():
            posGC, negGC = gc_count(test_df)
            deltaGC = posGC - negGC
            content.append([protein_name, deltaGC, data_type])

results_df = pd.DataFrame(content, columns=["Protein", "DeltaGC", "DataType"])
results_df.to_csv("/home/mrkvrbl/Diplomka/Data/outputs/testsets_GC_counts_rbp24", sep='\t')

In [ ]:
import pandas as pd

df = pd.re

In [ ]:
import re

print(re.sub(r"(^chr[\w]+):(\d+)-(\d+)\(\)", r"\1\t\2\t\3", "chr16:2806320-2806448()"))

chr16	2806320	2806448


In [ ]:
import pandas as pd
exon = pd.read_csv("/home/mrkvrbl/Diplomka/Data/reference/region_types/exons_intersections.tsv", delimiter="\t", header=None).drop_duplicates().reset_index(drop=True)
exon.to_csv("/home/mrkvrbl/Diplomka/Data/reference/region_types/exons_intersections.tsv", sep="\t", index=None, header=None)

In [ ]:
intron = pd.read_csv("/home/mrkvrbl/Diplomka/Data/reference/region_types/introns_intersections.tsv", delimiter="\t", header=None).drop_duplicates().reset_index(drop=True)
intron.to_csv("/home/mrkvrbl/Diplomka/Data/reference/region_types/introns_intersections.tsv", sep="\t", index=None, header=None)

In [ ]:
utr3 = pd.read_csv("/home/mrkvrbl/Diplomka/Data/reference/region_types/utr3_intersections.tsv", delimiter="\t", header=None).drop_duplicates().reset_index(drop=True)
utr3.to_csv("/home/mrkvrbl/Diplomka/Data/reference/region_types/utr3_intersections.tsv", sep="\t", index=None, header=None)

In [ ]:
utr5 = pd.read_csv("/home/mrkvrbl/Diplomka/Data/reference/region_types/utr5_intersections.tsv", delimiter="\t", header=None).drop_duplicates().reset_index(drop=True)
utr5.to_csv("/home/mrkvrbl/Diplomka/Data/reference/region_types/utr5_intersections.tsv", sep="\t", index=None, header=None)

### Prepare protein without mapping

In [ ]:
import gzip
import os
from pathlib import Path
from sklearn.utils import shuffle

In [ ]:
def get_sequences(path):
    sequences = []
    with gzip.open(path, "rb") as f_in:
        lines = f_in.readlines()
        for line in lines:
            line = line.decode("ascii")
            if not line.startswith(">"):
                sequences.append(line.lower().strip())
    return sequences

def save_data(data, path):
    with open(path, "w") as f_out:
        for entry in data:
            f_out.write(entry[0] + "\t" + str(entry[1]) + "\n")


In [ ]:
train_positives = get_sequences("/home/mrkvrbl/Diplomka/Data/rbp24/raw/CAPRIN1_Baltz2012.train.positives.fa.gz")
train_positives = [[seq, 1] for seq in train_positives]
train_negatives = get_sequences("/home/mrkvrbl/Diplomka/Data/rbp24/raw/CAPRIN1_Baltz2012.train.negatives.fa.gz")
train_negatives = [[seq, 0] for seq in train_negatives]

test_positives = get_sequences("/home/mrkvrbl/Diplomka/Data/rbp24/raw/CAPRIN1_Baltz2012.ls.positives.fa.gz")
test_positives = [[seq, 1] for seq in test_positives]
test_negatives = get_sequences("/home/mrkvrbl/Diplomka/Data/rbp24/raw/CAPRIN1_Baltz2012.ls.negatives.fa.gz")
test_negatives = [[seq, 0] for seq in test_negatives]

train = shuffle(train_negatives + train_positives)
test = shuffle(test_negatives + test_positives)

save_data(train, "/home/mrkvrbl/Diplomka/Data/rbp24/extracted_seqs/train")
save_data(test, "/home/mrkvrbl/Diplomka/Data/rbp24/extracted_seqs/test")

In [ ]:
seq_lens = [len(sequence[0]) for sequence in train+test]
min_len, max_len = min(seq_lens), max(seq_lens)
print(f"Min lenght: {min_len}\nMax lenght: {max_len}")

In [ ]:
entry = train[0]
entry

['caccucguagaggcgagccacauguuacucgucguaccccagacuacuuccuguaaugaaauuuuaaacuuguacaguauugccaugaaccauauaucgaccuaauggaaaugggaagagcaacaguaacuccaaagugucagaaaauaguuaacauucaaaaaacuuguuuucacauggaccaaaagaugugccauauaaaaauacaaagccucuugucaucaacagccgugaccacuuuagaaugaaccaguucauugcaugcugaagcgacauuguuggucaagaaaccaguuucuggcauagcgcuauuuguaguuacuuuugcuuucucugag',
 1]

In [ ]:
from Bio import SeqIO
import gzip
path_to_file = "/home/mrkvrbl/Diplomka/Data/rbp31/CLIPSEQ_AGO2/test/sequences.fa.gz"
labels = [record.description.split(";")[-1][-1] for record in SeqIO.parse(gzip.open(path_to_file, "rt"), 'fasta')]
seqs = [[str(record.id), str(record.seq), record.description.split(";")[-1][-1]] for record in SeqIO.parse(gzip.open(path_to_file, "rt"), 'fasta')]
seqs

[['chr1,+,900921,901021;',
  'CCCAGCTCGAATTTTGCACATGGCGGGGTCCCGGGAAGGGTGGGGAGCAGTTGTCCTTCCTGTCGTCGTCTGCCGTGTGCCATCTTTCCTGGATCTTGTAG',
  '0'],
 ['chr1,+,900985,901085;',
  'GTCGTCTGCCGTGTGCCATCTTTCCTGGATCTTGTAGTGGGTGCACACGCGTGCACTGGGACCCCACACAGCAATACGAGTCCAACTTAATAAACACATTT',
  '0'],
 ['chr1,+,1336456,1336556;',
  'CCCTGGCCTCCAAAGGCTTAGGGCGTAGTGGCTGTCATGAGGCACAGTGACAAGGGCCAGGAGGGGGACCTCTTAGGGGGCCATGCAGTGATGGAAGCCAG',
  '0'],
 ['chr1,+,1846801,1846901;',
  'CCTCTGGGAAGGTAGACTTGGAGCTTGGTCCAGTGAAGAGGGAACAGCACGGGGAACAGCGTGTGTGAGGGTCCCGCCATGTCTCAGCCCTTTCCTGGCCC',
  '0'],
 ['chr1,+,1847981,1848081;',
  'CTCATGAGCCTGCTGGGTATCAACCCCACCAAGAGTGAGCTGGCCTCAATGGCCAAGGATGTGGACAGAGACAGTGAGCTCATGGCTGGAGTGGGGTGGGC',
  '0'],
 ['chr1,+,1953396,1953496;',
  'AGGTGACCCATGGAGAGCCACCCGAGGGGGAAGCTGCCATGTCTTCATGGTGCCCACTCTGCCCCCTGAGTTTGGAGTCCTAAATCTGAGTGTAGACTTGG',
  '0'],
 ['chr1,+,2103704,2103804;',
  'ACAAGAACCCTCTCCCAGTAACTTTGCCCCCACAGGAAGGCCTGGGCCCTGGTGACACAACGAGCACTTTCTGCGGAACCCCGAATTACATCGCCCCCGAA',
  '1

In [ ]:
import pandas as pd

In [ ]:
original = pd.read_csv("/home/mrkvrbl/Diplomka/Data/rbp24/extracted_seqs/ALKBH5_Baltz2012/train/original.tsv", delimiter='\t', names=['seq', 'label'])
original

,seq,label
0,ggcccgcagccgcatcctcagggcagggggcaagatcctcactttc...,0
1,gaagctttgccacatctgtactcacagttcatgtcacttgtgactt...,1
2,cccaagcgcgggcggctccagctccgggacgacgaccacgacgacg...,1
3,tgctgccctcatgcttagaaaacaggaagtggtaggtcgggcgcgg...,0
4,atgattgccattctaactggcatgagatggtatctcattgtggttt...,0
...,...,...
2405,aaaaaaaaggatcaaatgaatcagttttgaactaatccacttaaat...,0
2406,ctgatgagcctttgtagctaactcatttttaccctgttctgctgtc...,0
2407,ttttaaggacttgattacatcattttcaagcctgatagttttggaa...,1
2408,caccacctctcttcagtaaagttgttattgttctgatagatgcctt...,1


# N E G A T I V E S

In [1]:
def intersection(a, b):

    a = pybedtools.BedTool(a)
    b = pybedtools.BedTool(b)

    return a.intersect(b, u=True, f=0.51, wa=True)

In [2]:
def sample(pos, neg):

    if len(pos) >= len(neg):
        return neg
    
    return neg.sample(n = max(1, len(pos)))

In [4]:
import pybedtools
from pathlib import Path
import pandas as pd
import glob

INTRONS = "/home/mrkvrbl/Diplomka/Data/reference/region_types/introns.bed.gz"
EXONS = "/home/mrkvrbl/Diplomka/Data/reference/region_types/exons.bed.gz"
UTR5 = "/home/mrkvrbl/Diplomka/Data/reference/region_types/5utrs.bed.gz"
UTR3 = "/home/mrkvrbl/Diplomka/Data/reference/region_types/3utrs.bed.gz"
REPEATS = "/home/mrkvrbl/Diplomka/Data/reference/repeat_masker_hg19.gz"

positives = pybedtools.BedTool(Path("/home/mrkvrbl/Diplomka/Data/rbp24/processed/CLIPSEQ_ELAVL1/test/positives.bed"))
seed = 1 # so this test always returns the same results

#generate negaatives by shuffling the coordinates of possitives in each region separately
introns = positives.shuffle(genome='hg19', chrom=True, seed=seed, noOverlapping=True, incl=INTRONS)
exons = positives.shuffle(genome='hg19', chrom=True, seed=seed, noOverlapping=True, incl=EXONS)
utr5 = positives.shuffle(genome='hg19', chrom=True, seed=seed, noOverlapping=True, incl=UTR5)
utr3 = positives.shuffle(genome='hg19', chrom=True, seed=seed, noOverlapping=True, incl=UTR3)

#intersect postivies with each region
pos_intron_intersect = intersection(positives, INTRONS)
pos_exon_intersect = intersection(positives, EXONS)
pos_utr3_intersect = intersection(positives, UTR3)
pos_utr5_intersect = intersection(positives, UTR5)
pos_repeat_intersect = intersection(positives, REPEATS)

# get repeat and non repeat sequences from each region of positive sequences
pos_intron_repeat = intersection(pos_intron_intersect, pos_repeat_intersect)
pos_intron_nonrepeat = pos_intron_intersect - pos_intron_repeat

pos_exon_repeat = intersection(pos_exon_intersect, pos_repeat_intersect)
pos_exon_nonrepeat = pos_exon_intersect - pos_exon_repeat

pos_utr3_repeat = intersection(pos_utr3_intersect, pos_repeat_intersect)
pos_utr3_nonrepeat = pos_utr3_intersect - pos_utr3_repeat

pos_utr5_repeat = intersection(pos_utr5_intersect, pos_repeat_intersect)
pos_utr5_nonrepeat = pos_utr5_intersect - pos_utr5_repeat

# get repeat and nonrepeat regions of generated negative sequences
neg_intron_repeat = intersection(introns, REPEATS)
neg_intron_nonrepeat = introns - neg_intron_repeat

neg_exon_repeat = intersection(exons, REPEATS)
neg_exon_nonrepeat = exons - neg_exon_repeat

neg_utr3_repeat = intersection(utr3, REPEATS)
neg_utr3_nonrepeat = utr3 - neg_utr3_repeat

neg_utr5_repeat = intersection(utr5, REPEATS)
neg_utr5_nonrepeat = utr5 - neg_utr5_repeat


#sample the generated sequences acording to the positive sequences distribution
sampled_neg_intron_repeat = sample(pos_intron_repeat, neg_intron_repeat)
sampled_neg_intron_nonrepeat = sample(pos_intron_nonrepeat, neg_intron_nonrepeat)

sampled_neg_exon_repeat = sample(pos_exon_repeat, neg_exon_repeat)
sampled_neg_exon_nonrepeat = sample(pos_exon_nonrepeat, neg_exon_nonrepeat)

sampled_neg_utr3_repeat = sample(pos_utr3_repeat, neg_utr3_repeat)
sampled_neg_utr3_nonrepeat = sample(pos_utr3_nonrepeat, neg_utr3_nonrepeat)

sampled_neg_utr5_repeat = sample(pos_utr5_repeat, neg_utr5_repeat)
sampled_neg_utr5_nonrepeat = sample(pos_utr5_nonrepeat, neg_utr5_nonrepeat)

#put it together

negatives = {"neg_intron_repeat":sampled_neg_intron_repeat, 
            "neg_intron_nonrepeat":sampled_neg_intron_nonrepeat,
            "neg_exon_repeat":sampled_neg_exon_repeat,
            "neg_exon_nonrepeat":sampled_neg_exon_nonrepeat, 
            "neg_utr5_repeat":sampled_neg_utr5_repeat, 
            "neg_utr5_nonrepeat":sampled_neg_utr5_nonrepeat, 
            "neg_utr3_repeat":sampled_neg_utr3_repeat, 
            "neg_utr3_nonrepeat":sampled_neg_utr3_nonrepeat}

In [8]:
dfs = []
for value in negatives.values():
    dfs.append(value.to_dataframe())

df = pd.concat(dfs)
df.to_csv

,chrom,start,end,name
0,chr4,95777931,95777973,+
1,chr6,157241989,157242047,+
2,chr13,50766871,50766911,+
3,chr2,201891227,201891270,-
4,chr13,36200055,36200095,+
...,...,...,...,...
398,chr1,55353496,55353539,-
399,chr2,37870897,37870937,+
400,chrX,106174074,106174121,+
401,chr2,111923664,111923706,+


In [129]:

print(len(pos_intron_repeat) + len(pos_intron_nonrepeat) + len(pos_exon_repeat) + len(pos_exon_nonrepeat) + len(pos_utr5_repeat) + len(pos_utr5_nonrepeat) + len(pos_utr3_repeat) + len(pos_utr3_nonrepeat))

588


In [130]:
print(len(sampled_neg_intron_repeat) + len(sampled_neg_intron_nonrepeat) + len(sampled_neg_exon_repeat) + len(sampled_neg_exon_nonrepeat) + len(sampled_neg_utr5_repeat) + len(sampled_neg_utr5_nonrepeat) + len(sampled_neg_utr3_repeat) + len(sampled_neg_utr3_nonrepeat))

580


In [131]:
def adjust_window(line, window_size):
    #Adjust coordinate window of a bed file
    start = line.start
    end = line.stop
    length = end - start
    middle = start + (length / 2)
    
    new_start = middle - (window_size/2)
    new_end = middle + (window_size/2)
    
    line.start = new_start
    line.stop = new_end

    return line

In [132]:
negatives = {"neg_intron_repeat":sampled_neg_intron_repeat, 
            "neg_intron_nonrepeat":sampled_neg_intron_nonrepeat,
            "neg_exon_repeat":sampled_neg_exon_repeat,
            "neg_exon_nonrepeat":sampled_neg_exon_nonrepeat, 
            "neg_utr5_repeat":sampled_neg_utr5_repeat, 
            "neg_utr5_nonrepeat":sampled_neg_utr5_nonrepeat, 
            "neg_utr3_repeat":sampled_neg_utr3_repeat, 
            "neg_utr3_nonrepeat":sampled_neg_utr3_nonrepeat}

In [133]:
path = "/home/mrkvrbl/Diplomka/Data/rbp24/processed/CLIPSEQ_ELAVL1/test/"

for bed_file_name, bed_tool_obj in negatives.items():
    bed_file_out = path + bed_file_name + ".tsv"
    entries = bed_tool_obj.sort().each(adjust_window, 101).sequence(fi="/home/mrkvrbl/Diplomka/Data/reference/referecnce_genome/hg19_latest.fa", s=True, tab=True)
    entries.save_seqs(bed_file_out)

In [134]:
import pandas as pd
import glob
import shutil

tsv_files = glob.glob("/home/mrkvrbl/Diplomka/Data/rbp24/processed/CLIPSEQ_ELAVL1/test/neg_*repeat.tsv")
dfs = []
for file in tsv_files:
    df = pd.read_csv(file,delimiter="\t", header=None, names=['id', 'seq'])
    df['label'] = 0
    dfs.append(df)
    os.remove(file)

neg_df = pd.concat(dfs)

In [135]:
orig_df = pd.read_csv("/home/mrkvrbl/Diplomka/Data/rbp24/processed/CLIPSEQ_ELAVL1/test/original.tsv.gz", delimiter="\t", index_col=0, compression="gzip")
pos_df = orig_df[orig_df.label == 1]
old_neg_df = orig_df[orig_df.label == 0]

In [160]:
def get_gc_count_from_seq(seq):
    gc = 0
    count = 0

    if type(seq) == "str":
        seq = seq.lower()
        for char in seq:
            count += 1
            if char == "g" or char == "c":
                gc += 1
            
        return gc * 100 / count

    seqs = seq

    for seq in seqs:
        seq = seq.lower()
        for char in seq:
            count += 1
            if char == "g" or char == "c":
                gc += 1

    return gc * 100 / count

In [161]:
halo = "halo"
type(halo)

str

In [162]:
new_neg_gc = get_gc_count_from_seq(neg_df.seq)
pos_gc = get_gc_count_from_seq(pos_df.seq)
old_neg_gc = get_gc_count_from_seq(old_neg_df.seq)

print(pos_gc, new_neg_gc, old_neg_gc)

31.748514851485147 40.82772277227723 42.95049504950495


In [163]:
def get_gc_count_from_seq(seq):
    gc = 0
    count = 0

    seq = seq.lower()
    for char in seq:
        count += 1
        if char == "g" or char == "c":
            gc += 1

    return gc * 100 / count

In [164]:
pos_df['GC'] = pos_df.seq.apply(lambda x: get_gc_count_from_seq(x))
pos_df.sort_values("GC", ascending=False).reset_index(drop=True)

/home/mrkvrbl/miniconda3/envs/diplomka/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,seq,label,GC
0,chr11:62349003-62349104(),TGGTGACTGTCGGGGGCCGCTCCTTTGGGGGATTTGGAGGCCGGGC...,1,67.326733
1,chr14:23795076-23795177(),GTAGCCGGAAGATCTCCGTTCCGCTCCCAGCGGCTCCAGTGTAAAT...,1,56.435644
2,chr5:154196760-154196861(),GGAACCCCAGAGTACCTCCTGTGTGGAAGGGTCCCTGGATTTTCCC...,1,54.455446
3,chr14:20811498-20811599(),AAGGGACATGGGAGTGGAGTGACAGGACGCACTCAGCTCGTGGCCC...,1,54.455446
4,chr22:20098835-20098936(),TTCTGCCGTGTCTGTCCCCCACCATGCCCCCTACAGGCGGTACTGA...,1,54.455446
...,...,...,...,...
495,chr1:36770463-36770564(),TTATGAATTCTATTATCTTTACAAATATGAGaaaattttttcaata...,1,17.821782
496,chr1:222731840-222731941(),TTGTTTCTCTAGCtataaataatacaaaaatataaaaatatataaa...,1,16.831683
497,chr1:154323526-154323627(),TTTTATACCAAAGAGGAAATGATTTTTTTTCATATTTTGTTTGTCt...,1,14.851485
498,chr3:191111962-191112063(),AATTTGTTACTTTTTTTAAAGGtttttaaaagattacaattttaaa...,1,13.861386


In [140]:
neg_df['GC'] = neg_df.seq.apply(lambda x: get_gc_count_from_seq(x))
neg_df = neg_df.sort_values("GC", ascending=False).reset_index(drop=True)
neg_df = neg_df.iloc[80:]
neg_df

,id,seq,label,GC
80,chr3:44959506-44959607(),TGGCTGACACGGTCTGGAGAAGGGGTTGAACCAGTTCAGGGAGAAG...,0,58.415842
81,chr12:2692556-2692657(),CTGCTCTCCAGGACTTTAGTCACGGGCAGAGCTGCACAGACTTCAA...,0,58.415842
82,chr5:60993753-60993854(),tcccagtgtatttagaataaaagccaagtaccgtcccacggcctcc...,0,58.415842
83,chr1:27627699-27627800(),CTTCCCTGCCTTCCCAGGGATGGTGACCACTATGATGCCCTGAGGG...,0,58.415842
84,chr12:124969343-124969444(),tcctggccacctcccccacccatgaatgaacgcagggtggttcacc...,0,57.425743
...,...,...,...,...
575,chr1:210334637-210334738(),AATTTAATAAGATGTTTTTGATTTGAAGTTAATTTTAATTTAGCAA...,0,19.801980
576,chr9:127714020-127714121(),AAAAAGAAAAATCTTTATAATATACACTGAGTCCTATTTAAGATAA...,0,19.801980
577,chr1:47764318-47764419(),CTAAATAAAATACAAAGAATAATACAGAATATACAATATAGAACAC...,0,18.811881
578,chr4:178352657-178352758(),TTCAGATAATATAAGAAAGGTTAAATAAAAATAGATAATACAATTT...,0,18.811881


In [141]:
new_neg_gc = get_gc_count_from_seqs(neg_df.seq)
pos_gc = get_gc_count_from_seqs(pos_df.seq)
old_neg_gc = get_gc_count_from_seqs(old_neg_df.seq)

print(pos_gc, new_neg_gc, old_neg_gc)

31.748514851485147 40.82772277227723 42.95049504950495


In [142]:
df = pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)
df

,id,seq,label,GC
0,chr5:53840438-53840539(),TTTAAAAAAAAAAAAGGTATTGATGAGCCCCCCCCCCCCAGGACAT...,1,28.712871
1,chr1:113499108-113499209(),CGAGCTCTAGGCAATCTGCCCTCGTTCGGGACACAACCATCGGTCT...,0,56.435644
2,chr9:127714020-127714121(),AAAAAGAAAAATCTTTATAATATACACTGAGTCCTATTTAAGATAA...,0,19.801980
3,chr2:242293350-242293451(),AATGTATTTTGAAAACACTTATTTTACACAGCAATTTTGTATCCAT...,1,21.782178
4,chr4:175239904-175240005(),ATTTGAAACAGTGTAAGAAATCACTTTTAAGAAAACATTTTTAGAA...,1,27.722772
...,...,...,...,...
995,chrX:106184801-106184902(),GAGGACAGAAGTAAGGAGATAGCTGACGTGGATACGTAGCCGCGTA...,0,52.475248
996,chr12:56431670-56431771(),ACATGGAGAAAAGAGGTGTTTCCTTTTATATTGCTATTCAAAATCA...,1,30.693069
997,chr1:167759936-167760037(),AAACTATCTCTAGTTTTTCAATGGAAATATATCAGCTAGGGAAAAA...,1,25.742574
998,chr12:98943221-98943322(),gatccactgcacccggccGGCATTATGATTTTGTGTACTCTTGAAA...,0,36.633663
